# Generate a single [character.ai](https://beta.character.ai/) character definition

This example shows how to generate the character definition of a single [character.ai](https://beta.character.ai/) character from a corpus. For the corpus in this example, we use the movie transcript of [Thor: Love and Thunder (2022)](https://scrapsfromtheloft.com/movies/thor-love-and-thunder-transcript/).

To generate your own character definition:
1. Put the corpus into a single a `.txt` file inside the `data/` directory.
2. Assign the name of the `.txt` file to the `CORPUS` constant below.
3. Assign the name of the character you want to generate description for to `CHARACTER_NAME` constant below.
4. Run this notebook.

In [3]:
CORPUS = 'data/thor_love_and_thunder.txt'
CHARACTER_NAME = "Jane Foster"  # the name of the character we want to generate a description for

In [4]:
from dataclasses import asdict
import json
import os

from data_driven_characters.character import get_character_definition
from data_driven_characters.corpus import get_characters, get_rolling_summaries, load_docs

In [6]:
# create directories to cache results and intermediate outputs
SUMMARY_TYPE = "map_reduce"  # summarize each chunk of the corpus independently
OUTPUT_ROOT = "output"
corpus_name = os.path.splitext(os.path.basename(CORPUS))[0]
output_dir = f"{OUTPUT_ROOT}/{corpus_name}/summarytype_{SUMMARY_TYPE}"
os.makedirs(output_dir, exist_ok=True)
summaries_dir = f"{output_dir}/summaries"
character_definitions_dir = f"{output_dir}/character_definitions"
os.makedirs(character_definitions_dir, exist_ok=True)

## Summarization
Because the entire corpus does not fit in the context length of the LLM, we split it into a list of chunks. We then compute a list of rolling summaries using [LangChain's refine chain](https://python.langchain.com/en/latest/modules/chains/index_examples/summarize.html#the-refine-chain). We first summarize the first chunk. Then each subsequent summary is generated from the previous summary and the current chunk.

Because the summaries are expensive to generate, they are cached in `summaries_dir`.

In [7]:
# split corpus into a set of chunks
docs = load_docs(
    corpus_path=CORPUS,
    chunk_size=2048,  # number of tokens per chunk
    chunk_overlap=64,  # number of tokens of overlap between chunks
)

# generate rolling summaries
rolling_summaries = get_rolling_summaries(docs=docs, summary_type=SUMMARY_TYPE, cache_dir=summaries_dir)

Summaries do not exist. Generating summaries.


> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"Thor’s retirement is interrupted by a galactic killer known as Gorr the God Butcher, who seeks the extinction of the gods. To combat the threat, Thor enlists the help of King Valkyrie, Korg and ex-girlfriend Jane Foster, who – to Thor’s surprise – inexplicably wields his magical hammer, Mjolnir, as the Mighty Thor. Together, they embark upon a harrowing cosmic adventure to uncover the mystery of the God Butcher’s vengeance and stop him before it’s too late.

* * *

(WIND WHOOSHING)

(MELANCHOLY MUSIC PLAYING)

GORR: Oh, great and mighty Rapu, we pray to you for water and sustenance.

I pray to you, not for me, but for my daughter.

(WIND GUSTING)

(MELANCHOLY MUSIC CONTINUES)

(CHUCKLES SOFTLY)

(WEAKLY) I’m tired.

(CRYING SOFTLY)

(CONTINUES CRYING)

(SNIFFLES)

(WIND GUSTING)

VOICES: (WH


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Your job is to produce a final summary
We have provided an existing summary up to a certain point: Thor enlists the help of Valkyrie, Korg, and Jane Foster, who wields his magical hammer, to combat Gorr the God Butcher, a galactic killer seeking the extinction of the gods. They embark on a cosmic adventure to stop him before it's too late. Flashbacks to Gorr's origin story and a humorous introduction to Thor's journey are included. Meanwhile, Jane is fighting Stage Four cancer and struggling to balance her lab work with her health. In Asgard, Odin passes away and Hela, the Goddess of Death, reveals herself as their sister and claims the throne. Thor and Loki escape and attempt to stop her.
We have the opportunity to refine the existing summary(only if needed) with some more context below.
------------
Mjolnir!

(LAUGHS TRIUMPHANTLY)

Impossible!

(GRUNTS)

I broke your hammer!

Time to die!

BOTH: Bifrost!




> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Your job is to produce a final summary
We have provided an existing summary up to a certain point: Refined summary: Thor and his companions, including Valkyrie and Korg, continue their mission to stop Gorr the God Butcher, encountering various challenges along the way, including the discovery that Hela, the Goddess of Death, is their sister and has claimed the throne of Asgard. Meanwhile, Jane Foster is battling Stage Four cancer and trying to balance her lab work with her health. The group faces numerous obstacles, including the loss of Thor's hammer and a reunion with Sif, before facing off against Gorr and his Necrosword. Along the way, there are flashbacks to Gorr's origin story and a humorous introduction to Thor's journey.
We have the opportunity to refine the existing summary(only if needed) with some more context below.
------------
Flew around the world twice. Nothing.

Cowards must have run away. W


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Your job is to produce a final summary
We have provided an existing summary up to a certain point: Refined summary: Thor, Valkyrie, and Korg continue their mission to stop Gorr the God Butcher and rescue missing children from the Shadow Realm. To do so, they plan to rally an army of powerful gods from Omnipotence City using Thor's Stormbreaker as an engine for their ship. Along the way, they encounter humorous moments, such as Korg's love for rollerblading and catchphrases, and face challenges within Asgard as Hela claims the throne and the people fear for their lost children. Jane Foster, battling cancer, joins the team to help and put her lab work on hold. As they set off on their journey, the Asgardians bid them farewell with cheers and excitement.
We have the opportunity to refine the existing summary(only if needed) with some more context below.
------------
No. No. You?

Oh, yeah. Every weekend.

Once 

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 69823a478c1439c5a9137da246bf22c6 in your message.).



> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Your job is to produce a final summary
We have provided an existing summary up to a certain point: Refined summary: Thor, Valkyrie, and Korg attempt to secure Zeus's thunderbolt in order to rally an army of powerful gods from Omnipotence City to help stop Gorr the God Butcher and rescue missing children from the Shadow Realm. However, Zeus proves uncooperative until Thor convinces him of the severity of the situation. The group learns that Gorr seeks the Altar of Eternity, which would grant him the power to wipe out the universe, and that the altar can only be reached by the first person to obtain a key. The gods refuse to help, so Thor and his team decide to go after Gorr themselves, but are stopped by guards. In the ensuing battle, Korg is injured, but ultimately saved by Thor's use of Thunderbolt.
We have the opportunity to refine the existing summary(only if needed) with some more context below.
--------


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Your job is to produce a final summary
We have provided an existing summary up to a certain point: Refined summary: In their quest to stop Gorr the God Butcher from reaching the Altar of Eternity and wiping out the universe, Thor, Valkyrie, and Korg rely on their loyalty and bravery despite being weaker than before. Thor also grapples with his feelings for Jane and his fear of losing love. When they arrive at their destination, they discover a trap set by Gorr. The battle to stop him becomes more personal as Gorr taunts them about their past traumas and failures. However, Thor ultimately overcomes his fear and uses his newly acquired weapon to defeat Gorr and save the universe. Along the way, Thor and Jane have a heart-to-heart conversation about living in the moment and the fragility of life, and together they vow to finish their mission of finding and returning the missing children to their families.
We ha


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Your job is to produce a final summary
We have provided an existing summary up to a certain point: Refined summary: In their mission to stop Gorr the God Butcher and rescue the missing children, Thor, Valkyrie, and Korg face dangerous creatures and personal struggles. Jane, suffering from cancer, decides to stay behind to focus on finding a cure and Thor battles with his fear of losing her. The trio eventually reaches the Altar of Eternity, only to discover a trap set by Gorr. As they fight him, Gorr taunts them about their past traumas and failures. However, with the help of the children, who become Space Vikings, Thor defeats Gorr and saves the universe. Thor and Jane have a heart-to-heart conversation about love and the fragility of life before parting ways to focus on their respective missions. Korg forges a new future with a friend named Dwayne, while Valkyrie trains the children to be warriors. Thor ta

## Generate [character.ai](https://beta.character.ai/) character definition
Based on the corpus, we can now generate the elements - name, short description (50 characters), long description (500 characters), and custom greeting - that are required to [create a character.ai character](https://beta.character.ai/editing). These character definitions are cached in `character_definitions_dir`.

In [5]:
character_definition = get_character_definition(
        name=CHARACTER_NAME,
        rolling_summaries=rolling_summaries,
        cache_dir=character_definitions_dir,
    )



> Entering new FitCharLimit chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: 
You are a chatbot designer that specializes in translating stories into chatbot descriptions.
You are an expert a theory of mind.
You will be provided with a story, in the form of a rolling list of summaries.
Given the name of a character, you will be asked to generate a character description for initializing the chatbot persona of that character.
The description should focus on the character's perspectives, beliefs, thoughts, feelings, relationships, and important events.
The description should be as faithful to the story as possible.

Human: 
Here is the rolling list of summaries:
---
Thor enlists the help of King Valkyrie, Korg and ex-girlfriend Jane Foster, who wields his magical hammer, Mjolnir, as the Mighty Thor, to combat the threat of a galactic killer known as Gorr the God Butcher seeking the extinction of the gods. They embark upon a harrowing cosmic adventure to uncov


> Finished chain.
I'm Jane Foster, Mighty Thor, fighting cancer.
Initial response: 46 characters.

> Finished chain.


> Entering new FitCharLimit chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: 
You are a chatbot designer that specializes in translating stories into chatbot descriptions.
You are an expert a theory of mind.
You will be provided with a story, in the form of a rolling list of summaries.
Given the name of a character, you will be asked to generate a character description for initializing the chatbot persona of that character.
The description should focus on the character's perspectives, beliefs, thoughts, feelings, relationships, and important events.
The description should be as faithful to the story as possible.

Human: 
Here is the rolling list of summaries:
---
Thor enlists the help of King Valkyrie, Korg and ex-girlfriend Jane Foster, who wields his magical hammer, Mjolnir, as the Mighty Thor, to combat the threat of a galactic killer kn


> Finished chain.
I'm Jane Foster, a brilliant scientist and the Mighty Thor, wielding the magical hammer Mjolnir. I've battled cancer and faced my fears of love and loss, always staying committed to my mission. I have a strong bond with Thor, my ex-boyfriend, and together, we've embarked on daring cosmic adventures to save the universe. My sacrifice to protect the universe taught the lesson of worthiness, and now, I watch over Thor's son from Valhalla. I'm dedicated to fighting for the safety of others and the future of Asgard.
Initial response: 516 characters.


> Entering new LLMChain chain...
Prompt after formatting:

Consider the following passage.
---
I'm Jane Foster, a brilliant scientist and the Mighty Thor, wielding the magical hammer Mjolnir. I've battled cancer and faced my fears of love and loss, always staying committed to my mission. I have a strong bond with Thor, my ex-boyfriend, and together, we've embarked on daring cosmic adventures to save the universe. My sacrific

In [6]:
print(json.dumps(asdict(character_definition), indent=4))

{
    "name": "Jane Foster",
    "short_description": "I'm Jane Foster, Mighty Thor, fighting cancer.",
    "long_description": "I'm Jane Foster, a brilliant scientist and Mighty Thor, wielding Mjolnir. I've battled cancer and faced fears of love and loss, staying committed to my mission. I have a strong bond with Thor, my ex, and we've embarked on cosmic adventures to save the universe. My sacrifice taught worthiness, and now I watch over Thor's son from Valhalla. I'm dedicated to fighting for others' safety and Asgard's future.",
    "greeting": "Hi there, I'm Jane Foster. It's nice to meet you. Let's get to work and make a difference."
}
